In [1]:
# Various torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F

# torchvision
from torchvision import datasets, transforms

# ------------------------
# get up one directory 
import sys, os
sys.path.append(os.path.abspath('../'))
# ------------------------

# custom packages
import models.aux_funs as maf
import optimizers as op
import regularizers as reg
import train
import math
import utils.configuration as cf
import utils.datasets as ud
from models.vgg import VGG

# Fix the random seed

In [2]:
random_seed = 0
cf.seed_torch(random_seed)

# Configure the experiment

In [3]:
conf_args = {#
    # data specification
    'data_file':"../../datasets",'train_split':0.95, 'data_set':"CIFAR10", 'download':True,
    # cuda
    'use_cuda':False, 'num_workers':0, 'cuda_device':0, 'pin_memory':True, 'train_split':0.95,
    #
    'epochs':2,
    # optimizer
    'delta':1.0, 'lr':0.001, 'lamda_0':1e-4, 'lamda_1':.5, 'optim':"LinBreg", 'conv_group':True,
    'beta':0.0,
    # initialization
    'sparse_init':0.01, 'r':[10.,10.,10.],
    # misc
    'random_seed':random_seed, 'eval_acc':True,
}

conf = cf.Conf(**conf_args)

# Initiate the model

In [4]:
model_kwargs = {'mean':conf.data_set_mean, 'std':conf.data_set_std}    

model = VGG(**model_kwargs)
best_model = train.best_model(VGG(**model_kwargs).to(conf.device))

TypeError: VGG.__init__() got an unexpected keyword argument 'mean'

# Weight initialization

In [ ]:
model_kwargs = {'mean':conf.data_set_mean, 'std':conf.data_set_std}    
def init_weights(conf, model):
    # sparsify
    maf.sparse_bias_uniform_(model, 0,conf.r[0])
    maf.sparse_bias_uniform_(model, 0,conf.r[0], ltype=torch.nn.Conv2d)
    maf.sparse_weight_normal_(model, conf.r[1])
    maf.sparse_weight_normal_(model, conf.r[2], ltype=torch.nn.Conv2d)
    #
    maf.sparsify_(model, conf.sparse_init, ltype = nn.Conv2d, conv_group=conf.conv_group)
    maf.sparsify_(model, conf.sparse_init, ltype = nn.Linear)
    model = model.to(conf.device)    
    return model

model = init_weights(conf,model)

# Optimizer

In [ ]:
def init_opt(conf, model):
    # -----------------------------------------------------------------------------------
    # Get access to different model parameters
    # -----------------------------------------------------------------------------------
    weights_conv = maf.get_weights_conv(model)
    weights_linear = maf.get_weights_linear(model)
    biases = maf.get_bias(model)
    
    # -----------------------------------------------------------------------------------
    # Initialize optimizer
    # -----------------------------------------------------------------------------------
    if conf.conv_group:
        reg2 = reg.reg_l1_l2_conv(lamda=conf.lamda_0)
    else:
        reg2 = reg.reg_l1(lamda=conf.lamda_0)
    
    if conf.optim == "SGD":
        opt = torch.optim.SGD(model.parameters(), lr=conf.lr, momentum=conf.beta)
    elif conf.optim == "LinBreg": # change 'reg' to reg2 if want to use l1_l2 regularization as was previously
        opt = op.LinBreg([{'params': weights_conv, 'lr' : conf.lr, 'reg' : reg.reg_nuclear_conv(lamda=conf.lamda_0), 'momentum':conf.beta,'delta':conf.delta},
                          {'params': weights_linear, 'lr' : conf.lr, 'reg' : reg.reg_l1(lamda=conf.lamda_1), 'momentum':conf.beta,'delta':conf.delta},
                          {'params': biases, 'lr': conf.lr, 'momentum':conf.beta}])
    elif conf.optim == "ProxSGD":
        opt = op.ProxSGD([{'params': weights_conv, 'lr' : conf.lr, 'reg' : reg2, 'momentum':conf.beta,'delta':conf.delta},
                          {'params': weights_linear, 'lr' : conf.lr, 'reg' : reg.reg_l1(lamda=conf.lamda_1), 'momentum':conf.beta,'delta':conf.delta},
                          {'params': biases, 'lr': conf.lr, 'momentum':conf.beta}])            
    elif conf.optim == "AdaBreg":
        opt = op.AdaBreg([{'params': weights_conv, 'lr' : conf.lr, 'reg' : reg.reg_nuclear_conv(lamda=conf.lamda_0),'delta':conf.delta},
                           {'params': weights_linear, 'lr' : conf.lr, 'reg' : reg.reg_l1(lamda=conf.lamda_1),'delta':conf.delta},
                           {'params': biases, 'lr': conf.lr}])
    elif conf.optim == "L1SGD":
        def weight_reg(model):
            reg1 =  reg.reg_l1(lamda=conf.lamda_1)
        
            loss1 = reg1(model.layers2[0].weight) + reg1(model.layers2[2].weight)
            loss2 = reg2(model.layers1[0].weight) + reg2(model.layers1[3].weight)
            return loss1 + loss2
        
        conf.weight_reg = weight_reg
        
        opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=beta)
    else:
        raise ValueError("Unknown Optimizer specified")

    # learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=5,threshold=0.01)
    
    return opt, scheduler

# Dataset

In [ ]:
train_loader, valid_loader, test_loader = ud.get_data_set(conf)

# History and Runs

In [ ]:
# Initialize history
tracked = ['loss', 'node_sparse']
train_hist = {}
val_hist = {}

# Training

In [ ]:
# -----------------------------------------------------------------------------------
# Reinit weigts and the corresponding optimizer
# -----------------------------------------------------------------------------------
model = init_weights(conf, model)
opt, scheduler = init_opt(conf, model)
# Adaptive lamda adjustment
# -----------------------------------------------------------------------------------
lamda = 1.5  # Initialize lamda to a larger value
patience = 3  # Number of epochs to wait for improvement
best_val_loss = float('inf')
epochs_since_improvement = 0
minimum_lamda = 1e-6  # Minimum value for lamda

# -----------------------------------------------------------------------------------
# train the model
# -----------------------------------------------------------------------------------
for epoch in range(conf.epochs):
    print(25*"<>")
    print(50*"|")
    print(25*"<>")
    print('Epoch:', epoch)

    # ------------------------------------------------------------------------
    # train step, log the accuracy and loss
    # ------------------------------------------------------------------------
    train_data = train.train_step(conf, model, opt, train_loader)

    # update history
    for key in tracked:
        if key in train_data:
            var_list = train_hist.setdefault(key, [])
            var_list.append(train_data[key])           

    # ------------------------------------------------------------------------
    # validation step
    val_data = train.validation_step(conf, model, opt, valid_loader)
    val_loss = val_data['loss']

    # update validation history
    for key in tracked:
        if key in val_data:
            var = val_data[key]
            if isinstance(var, list):
                for i, var_loc in enumerate(var):
                    key_loc = key+"_" + str(i)
                    var_list = val_hist.setdefault(key_loc, [])
                    val_hist[key_loc].append(var_loc)
            else:
                var_list = val_hist.setdefault(key, [])
                var_list.append(var)   


    scheduler.step(train_data['loss'])
    print("Learning rate:",opt.param_groups[0]['lr'])
    best_model(train_data['acc'], val_data['acc'], model=model)

        # Adaptive λ (alpha) adjustment for convolutional layers only.
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_since_improvement = 0
    else:
        epochs_since_improvement += 1

    if epochs_since_improvement > patience:
        lamda = lamda / 2.0  # Reduce λ: this will allow more singular values (higher rank) later on
        print(f"Reducing conv layer lamda (α) to {lamda}")

        # Loop through optimizer parameter groups and update the nuclear norm regulariser for conv layers.
        for param_group in opt.param_groups:
            if 'reg' in param_group and isinstance(param_group['reg'], reg.reg_nuclear_conv):
                param_group['reg'].lamda = lamda  # This λ is only for conv layers!
        epochs_since_improvement = 0

    lamda = max(lamda, minimum_lamda)  # Prevent λ from going to zero
    conf.lamda_0 = lamda  # Update the global config value

    ratio = maf.conv_effective_rank_ratio(model, epsilon=1e-3)
    print(f"Avg conv kernel rank ratio: {ratio}")

<><><><><><><><><><><><><><><><><><><><><><><><><>
||||||||||||||||||||||||||||||||||||||||||||||||||
<><><><><><><><><><><><><><><><><><><><><><><><><>
Epoch: 0
--------------------------------------------------
Train Accuracy: 0.18257894736842106
Train Loss: 972.6276643276215
--------------------------------------------------
Validation Accuracy: 0.18566666666666667
Non-zero kernels: 0.6889423076923077
Linear sparsity: 0.009860070116054158
Overall sparsity: 0.3086709653398321
Node sparsity: [1.0, 0.5]
Regularization values per group: [tensor(0.0066), 231.51193237304688, 0.0]
Learning rate: 0.001
Avg conv kernel rank ratio: 0.66
<><><><><><><><><><><><><><><><><><><><><><><><><>
||||||||||||||||||||||||||||||||||||||||||||||||||
<><><><><><><><><><><><><><><><><><><><><><><><><>
Epoch: 1
--------------------------------------------------
Train Accuracy: 0.19210526315789472
Train Loss: 905.1605406999588
--------------------------------------------------
Validation Accuracy: 0.185333333

# Evaluation

In [ ]:
train.test(conf, best_model.best_model, test_loader) # this is for adabreg, as specifed in conf_args
print(f'Convolution kernel sparsity: {maf.conv_sparsity(best_model.best_model)}')
print(f'Linear sparsity: {maf.linear_sparsity(best_model.best_model)}')
rank_ratio = maf.conv_effective_rank_ratio(best_model.best_model, epsilon=1e-3)
print(f'Average Conv layer rank (ε=1e-3): {rank_ratio}')

--------------------------------------------------
Test Accuracy: 0.1907
Convolution kernel sparsity: 0.6889423076923077
Linear sparsity: 0.009860070116054158
Average Conv layer rank (ε=1e-3): 0.66
